In [14]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import imageio
import zipfile
import cv2
from scipy.spatial import distance as dist
from collections import defaultdict
from io import StringIO
from PIL import Image


In [26]:
total_frames=200
def getFrames(vidfile):
    vid = cv2.VideoCapture(vidfile)
    frames = []
    endframe = True
    t = 0
    while endframe:
        if t>total_frames:
            break
        endframe,frame = vid.read()
        
        #resize images 

        scale_percent = 20 # percent of original size
        width = int(frame.shape[1] * scale_percent / 100)
        height = int(frame.shape[0] * scale_percent / 100)
        dim = (width, height)

      # resize image
        frame = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)
        #print(frame)
        if endframe:
            frames.append(frame)
            #print("processing frame: ",t)
            t+=1
    return frames

In [34]:
def compute_dense_optical_flow(prev_image, current_image):
    old_shape = current_image.shape
    prev_image_gray = cv2.cvtColor(prev_image, cv2.COLOR_BGR2GRAY)
    current_image_gray = cv2.cvtColor(current_image, cv2.COLOR_BGR2GRAY)

    assert current_image.shape == old_shape

    hsv = np.zeros_like(prev_image)
    hsv[..., 1] = 255
    flow = None
    flow = cv2.calcOpticalFlowFarneback(prev=prev_image_gray,
                                      next=current_image_gray, flow=flow,
                                      pyr_scale=0.8, levels=15, winsize=5,
                                      iterations=10, poly_n=5, poly_sigma=0,
                                          flags=10)

    #   mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    #   hsv[..., 0] = ang * 180 / np.pi / 2
    #   hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)

    #   return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR) 
    return flow

In [28]:
images_data= getFrames('tests/thyang5.mov')

In [35]:
with imageio.get_writer('Optflow.gif', mode='I',fps=20) as gifwriter:
    for i in range(0,len(images_data)-1):#trying to get better optical flow by comparing distant frames,                                                                 #instead of consecutive 

        image = compute_dense_optical_flow(images_data[i],images_data[i+1])
        gifwriter.append_data(image)



Lossy conversion from float32 to uint8. Range [-184.11012268066406, 313.3209533691406]. Convert image to uint8 prior to saving to suppress this warning.


KeyError: 2